In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
# The reason why we need TF_IDF is that it helps in identifying certain terms in a document (in our case a dataset)
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [28]:
df = pd.read_csv('dataset.csv') #We will be loading it later
df=df.dropna()

In [29]:
# Since the scope of our project is smaller than similar projects, we will be setting limits
max_words = 5000
max_length = 100

tokenizer = Tokenizer(num_words=max_words)
# With this, we are trying to update the vocabulary based on the word frequency
tokenizer.fit_on_texts(df['title'])
# Since neural networks require inputs of the same shape, we need to make sure we have padding
sequences = tokenizer.texts_to_sequences(df['title'])
X = pad_sequences(sequences, maxlen=max_length)
y = to_categorical(df['label'])

In [30]:
# Here, we're going to split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# This is where we're trying to build the model
model = Sequential()
model.add(Embedding(max_words, 50, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# Softmax is a mathematical function that takes a vector of real numbers as input and transforms it into a probability  distribution
# Reference - https://deepai.org/machine-learning-glossary-and-terms/softmax-layer
model.add(Dense(2, activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# This is where we're trying to train the model
batch_size = 32
epochs = 10
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

# Here, we try to evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/10
18/18 [==============================] - 3s 54ms/step - loss: 0.6802 - accuracy: 0.6077 - val_loss: 0.6697 - val_accuracy: 0.6721
Epoch 2/10
18/18 [==============================] - 1s 40ms/step - loss: 0.6208 - accuracy: 0.7190 - val_loss: 0.6323 - val_accuracy: 0.7049
Epoch 3/10
18/18 [==============================] - 1s 40ms/step - loss: 0.4938 - accuracy: 0.7755 - val_loss: 0.5855 - val_accuracy: 0.6721
Epoch 4/10
18/18 [==============================] - 1s 39ms/step - loss: 0.3383 - accuracy: 0.8796 - val_loss: 0.4918 - val_accuracy: 0.7377
Epoch 5/10
18/18 [==============================] - 1s 39ms/step - loss: 0.1988 - accuracy: 0.9343 - val_loss: 0.3805 - val_accuracy: 0.7541
Epoch 6/10
18/18 [==============================] - 1s 40ms/step - loss: 0.1127 - accuracy: 0.9690 - val_loss: 0.3080 - val_accuracy: 0.7869
Epoch 7/10
18/18 [==============================] - 1s 40ms/step - loss: 0.0646 - accuracy: 0.9854 - val_loss: 0.3223 - val_accuracy: 0.8197
Epoch 8/10
18

In [32]:
# Make predictions on new data
new_data = ["The news headline (fake or not)"]
new_data_sequences = tokenizer.texts_to_sequences(new_data)
new_data_padded = pad_sequences(new_data_sequences, maxlen=max_length)
predictions = model.predict(new_data_padded)

# This is to print the predicted class probabilities
print(f'Predicted Probabilities: {predictions}')

# This is to convert predicted probabilities to class labels (fake or true)
predicted_classes = [1 if prob[1] > prob[0] else 0 for prob in predictions]
print(f'Predicted Classes: {predicted_classes}')

1/1 [==============================] - 0s 187ms/step
Predicted Probabilities: [[0.9165709  0.08342909]]
Predicted Classes: [0]
